In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [4]:
data_txt = open('./data_split/data_dir_dict_0614win.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [ ]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [ ]:
f_dir1 = "D:/OneDrive/Develop/Smart_Fishery/models/ICROS/Train_0/model.ckpt-2280.meta"
f_dir2 = "D:/OneDrive/Develop/Smart_Fishery/models/ICROS/Train_0/"

saver = tf.train.import_meta_graph(f_dir1)
batch = 32
ev_steps = int(ev_label.shape[0]/batch)
ev_acc = np.zeros(ev_label.shape[0])

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    #for op in tf.get_default_graph().get_operations():
        #print(op.name)
        #print(op.values())
        
    data_in = graph.get_tensor_by_name("fifo_queue_DequeueUpTo:1")

    #feed_dict={data_in:ev_data[0:10], label:ev_label[0:10]}
    keep_prob = graph.get_tensor_by_name("dropout/dropout/keep_prob:0")
    for i in range(ev_steps):
        if i < (ev_steps-1):
            feed_dict={data_in:ev_data[batch*i:batch*(i+1)], keep_prob:1.0}
            label = ev_label[batch*i:batch*(i+1)].reshape(batch)
        else:
            feed_dict={data_in:ev_data[batch*i:], keep_prob:1.0}
            temp = ev_label[batch*i:]
            label = temp.reshape(len(temp))
        
        prob = graph.get_tensor_by_name("softmax_tensor:0")
        classes = graph.get_tensor_by_name("ArgMax:0")
        
        
        result_pr = sess.run(prob, feed_dict=feed_dict)
        result_cl = sess.run(classes, feed_dict=feed_dict)
        
        print(result_pr)
        print(result_cl)
        print(label)

        acc = np.equal(result_cl, label).reshape((1,len(label)))*1
        
        print(acc)
        print(np.mean(acc))
        
        if i < (ev_steps-1):
            ev_acc[batch*i:batch*(i+1)] = acc
        else:
            ev_acc[batch*i:] = acc


In [ ]:
print(ev_acc[-1])
print(np.mean(ev_acc))

In [ ]:
final = np.array([0.9398, 0.9517, 0.9639, 0.9737, 0.9814, 0.9639, 0.9595, 0.8742, 0.9814, 0.9825])
print(np.mean(final))
print(np.std(final))

In [ ]:
fail_idx = np.where(ev_acc == 0)
print(fail_idx[0])

In [ ]:
"""
for i in range(len(fail_idx[0])):
    new_img = cv2.imread(tr_data_dir['test'][fail_idx[0][i]][0])
    
    save_path = "/models/ICROS-fail/T9_fail-%d.png" % i
    cv2.imwrite(save_path, new_img)
"""

In [ ]:
ind1 = [18, 46, 63, 83, 85, 92,109,166,182,196,224,280,282,310,362,373,376,381,400,408,
        424,428,439,470,486,503,508,510,554,555,563,568,618,626,642,649,654,657,665,670,
        674,704,707,708,722,746,748,752,760,780,797,822,852,872,885]

In [5]:
for i in range(len(tr_data_dir['test'])):
    new_img = cv2.imread(tr_data_dir['test'][i][0])
    
    save_path = "D:/dataset_full/ev_set/ev-%d.png" % i
    cv2.imwrite(save_path, new_img)